# Report

We use three models to predict our data.
1. Logistic Regression
2. Neural Network
3. Decision Forest

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import matplotlib.pyplot as plot
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from keras.utils import np_utils


Using TensorFlow backend.


Reading our processed data

In [3]:
pickle_in = open("Input.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("Output.pickle","rb")
Y = pickle.load(pickle_in)
# print(Y)

In [4]:
print(X.keys())

x = X.values
print(x.shape)
y = Y.values
print(y.shape)

Index(['order_number', 'add_to_cart_order', 'department_id',
       'days_since_prior_order', 'order_hour_of_day', 'order_dow',
       'product_id', 'aisle_id'],
      dtype='object')
(9462265, 8)
(9462265,)


#  Logistic Regression

In [5]:
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.3)
logiR = LogisticRegression(solver = 'lbfgs')
logiR.fit(xTrain, yTrain)

yPrediction = logiR.predict(xTest)


In [6]:
score = logiR.score(xTest, yTest)
print("The accuracy after logistic regression is = ", score*100)


The accuracy after logistic regression is =  69.65216227260558


After performing a test/train split, we use logistic Regression library of sklearn and estimate a model. That model gives us a 69.6 percent accuracy on the test data.
The solver used was lbfgs.
The variable being predicted is which product would be reordered against all the attributes.

# Neural Network

We made a network of 4 normal layers and 3 dropout layers.
The input was the 8 attributes we selected and the output was whether it will be reordered or not.
The drop out layer was added to avoid overtraining.
Batch Normalization normalizes the activations of the previous layer at each batch, i.e. applies a transformation that maintains the mean activation close to 0 and the activation standard deviation close to 1.
This helped improve accuracy by 30 percent.
The weights are attached. 
The highest accuracy was 72 percent.

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization


In [8]:
#Source = https://gist.github.com/stevhliu/2dad7416cfb9eec107f3e5748cadbb2b#file-basicnn-py
m1 = Sequential()
m1.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                          beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros'))
m1.add(Dense(64, activation='relu', input_dim=8))
m1.add(Dropout(0.25))
m1.add(Dense(64, activation='relu'))
m1.add(Dropout(0.25))
m1.add(Dense(64, activation='relu'))
m1.add(Dropout(0.25))
m1.add(Dense(64, activation='relu'))
m1.add(Dense(2, activation='softmax'))
print(m1.summary)
# compile the model
m1.compile(loss='categorical_crossentropy',
           optimizer='adam',
           metrics=['accuracy'])

# fit the model
y_binary = np_utils.to_categorical(yTrain)
print(xTrain.shape)
print(y_binary.shape)
m1.fit(xTrain, y_binary,
       epochs=1,
       batch_size = 128)

# evaluate the model
m1.evaluate(xTrain, y_binary, batch_size=128)


<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x0000018A4CC7A358>>
Instructions for updating:
Colocations handled automatically by placer.
(6623585, 8)
(6623585, 2)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
6623585/6623585 [==============================] - 66s 10us/step


[0.5415709088398101, 0.7228571838360478]

In [9]:
y_test = np_utils.to_categorical(yTest)
Results = m1.evaluate(xTest, y_test, batch_size=128)


2838680/2838680 [==============================] - 28s 10us/step


In [14]:
m1.save_weights('weights.h5')


In [15]:
print("The accuracy of the neural network is = ",  Results[1]*100)

The accuracy of the neural network is =  72.29543308863545


# Decision Trees


In [16]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestRegressor


In [17]:
regressor = RandomForestRegressor(n_estimators=10)  
regressor.fit(xTrain, yTrain)  
y_pred = regressor.predict(xTest)
print(y_pred)

[0.5 0.7 0.6 ... 0.4 0.9 0. ]


In [18]:
# ypred = np_utils.to_categorical(y_pred)
# print(map(round,y_pred))
roundedlist = list(map(round,y_pred))
# print()
# print(yTest)
print("The accuracy for the Decision Tree is = " , accuracy_score(yTest, roundedlist)*100) 


The accuracy for the Decision Tree is =  69.62450857440783


In [19]:
print(confusion_matrix(yTest,roundedlist))  
print(classification_report(yTest,roundedlist))  


[[ 734330  426438]
 [ 435825 1242087]]
              precision    recall  f1-score   support

           0       0.63      0.63      0.63   1160768
           1       0.74      0.74      0.74   1677912

   micro avg       0.70      0.70      0.70   2838680
   macro avg       0.69      0.69      0.69   2838680
weighted avg       0.70      0.70      0.70   2838680



The final model we used was a Decision forest.
We again used an SKlearn library and 10 decision trees were made. The accuracy was around 69 percent.
